# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
%run -i 'create_tables.py'

In [3]:
# %run -i 'sql_queries.py'

In [4]:
#!pip3 install psycopg2

In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files("data/song_data")

In [8]:
filepath = song_files[0]

In [9]:
df = pd.DataFrame([pd.read_json(filepath, typ='series', convert_dates=False)])
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARULZCI1241B9C8611,None,None,,Luna Orbit Project,SOSWKAV12AB018FC91,Midnight Star,335.51628,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [11]:
for value in df.values:
        num_songs, artist_id, artist_latitude, artist_longitude, artist_location, artist_name, song_id, title, duration, year = value

song_data = (song_id, title, artist_id, year, duration)
song_data

('SOSWKAV12AB018FC91', 'Midnight Star', 'ARULZCI1241B9C8611', 0, 335.51628)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
artist_data = (artist_id, artist_name, artist_location, artist_latitude, artist_longitude)
artist_data

('ARULZCI1241B9C8611', 'Luna Orbit Project', '', None, None)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files('data/log_data')

In [16]:
filepath = log_files[0]

In [17]:
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Adler,M,0,Barrera,NaN,free,"New York-Newark-Jersey City, NY-NJ-PA",GET,Home,1.540836e+12,248,None,200,1541470364796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,1541470383796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,1541470632796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
3,None,Logged In,Samuel,M,0,Gonzalez,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540493e+12,252,None,200,1541473967796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,1541474048796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df = df[df['page'] == "NextSong"].astype({'ts': 'datetime64[ms]'})
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,2018-11-06 02:13:03.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,2018-11-06 02:17:12.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,2018-11-06 03:14:08.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
5,The Gerbils,Logged In,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,(iii),200,2018-11-06 04:56:11.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
6,AFI,Logged In,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,Girl's Not Grey,200,2018-11-06 04:56:38.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55


In [19]:
time_df = pd.DataFrame()
time_df['timestamp'] = df['ts']
time_df['hour'] = df['ts'].dt.hour
time_df['day'] = df['ts'].dt.day
time_df['weekday'] = df['ts'].dt.weekday
time_df['month'] = df['ts'].dt.month
time_df['year'] = df['ts'].dt.year
time_df['week'] = df['ts'].dt.week
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
1,Gustavo Cerati,Logged In,Adler,M,1,Barrera,249.44281,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Uno Entre 1000,200,2018-11-06 02:13:03.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
2,Limp Bizkit,Logged In,Adler,M,2,Barrera,270.49751,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540836e+12,248,Behind Blue Eyes,200,2018-11-06 02:17:12.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",100
4,Mikel Erentxun,Logged In,Samuel,M,1,Gonzalez,178.83383,free,"Houston-The Woodlands-Sugar Land, TX",PUT,NextSong,1.540493e+12,252,Frases Mudas,200,2018-11-06 03:14:08.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",61
5,The Gerbils,Logged In,Martin,M,0,Johnson,27.01016,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,(iii),200,2018-11-06 04:56:11.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55
6,AFI,Logged In,Martin,M,1,Johnson,190.45832,free,"Minneapolis-St. Paul-Bloomington, MN-WI",PUT,NextSong,1.541082e+12,250,Girl's Not Grey,200,2018-11-06 04:56:38.796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",55


In [20]:
time_data = ()
column_labels = ()

In [21]:
time_df.head()

,timestamp,hour,day,weekday,month,year,week
1,2018-11-06 02:13:03.796,2,6,1,11,2018,45
2,2018-11-06 02:17:12.796,2,6,1,11,2018,45
4,2018-11-06 03:14:08.796,3,6,1,11,2018,45
5,2018-11-06 04:56:11.796,4,6,1,11,2018,45
6,2018-11-06 04:56:38.796,4,6,1,11,2018,45


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
len(time_df)

154

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId','firstName','lastName','gender','level']]

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    results = cur.execute(song_select,(row.song, row.artist, row.length))
    songid, artistid = results if results else None, None
    # insert songplay record
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row['ts'], row['userId'], row['level'], songid, artistid, row['sessionId'], 
                     row['location'], row['userAgent'])
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [27]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [28]:
%run -i 'etl.py'

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file